In [3]:
%%script bash
mkdir scrapers/
mkdir scrapers/amazon
cd scrapers/amazon
python -m pip install virtualenv
virtualenv env
. env/bin/activate
python -m pip install scrapy
python -m pip install numpy
python -m pip install pandas
python -m pip install requests
python -m pip install scraperapi-sdk
scrapy startproject amazon_scraper

  Created wheel for distlib: filename=distlib-0.3.0-py3-none-any.whl size=340427 sha256=00bcbc4e6f3a19398f3580eb2be82e635bccf84c378c05bf28685f101284d17d
  Stored in directory: /home/jovyan/.cache/pip/wheels/a2/19/da/a15d4e2bedf3062c739b190d5cb5b7b2ecfbccb6b0d93c861b
Successfully built distlib
created virtual environment CPython3.7.6.final.0-64 in 510ms
  creator CPython3Posix(dest=/home/jovyan/scrapers/amazon/env, clear=False, global=False)
  seeder FromAppData(download=False, pip=latest, setuptools=latest, wheel=latest, via=copy, app_data_dir=/home/jovyan/.local/share/virtualenv/seed-app-data/v1.0.1)
  activators BashActivator,CShellActivator,FishActivator,PowerShellActivator,PythonActivator,XonshActivator
  Created wheel for protego: filename=Protego-0.1.16-py3-none-any.whl size=7765 sha256=777005065fd2acbd1eecc394a36b0e705f7eea8d112d5043fc390d8d90183977
  Stored in directory: /home/jovyan/.cache/pip/wheels/ca/44/01/3592ccfbcfaee4ab297c4097e6e9dbe1c7697e3531a39877ab
  Created wheel f

In [4]:
%%script bash
python -m pip install pandas

  Using cached pandas-1.0.4-cp37-cp37m-manylinux1_x86_64.whl (10.1 MB)
  Using cached numpy-1.18.5-cp37-cp37m-manylinux1_x86_64.whl (20.1 MB)
  Using cached pytz-2020.1-py2.py3-none-any.whl (510 kB)


In [ ]:
import pandas as pd
data = pd.read_csv('store_credentials.csv', header=0, sep=';', index_col=False)
data

In [5]:
%%writefile settings.py
# -*- coding: utf-8 -*-

# Scrapy settings for amazon_scraper project
#
# For simplicity, this file contains only settings considered important or
# commonly used. You can find more settings consulting the documentation:
#
#     https://docs.scrapy.org/en/latest/topics/settings.html
#     https://docs.scrapy.org/en/latest/topics/downloader-middleware.html
#     https://docs.scrapy.org/en/latest/topics/spider-middleware.html

BOT_NAME = 'amazon_scraper'

SPIDER_MODULES = ['amazon_scraper.spiders']
NEWSPIDER_MODULE = 'amazon_scraper.spiders'


# Crawl responsibly by identifying yourself (and your website) on the user-agent
USER_AGENT = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.3 Safari/605.1.15'

# Obey robots.txt rules
ROBOTSTXT_OBEY = False

# Configure maximum concurrent requests performed by Scrapy (default: 16)
#CONCURRENT_REQUESTS = 32

# Configure a delay for requests for the same website (default: 0)
# See https://docs.scrapy.org/en/latest/topics/settings.html#download-delay
# See also autothrottle settings and docs
#DOWNLOAD_DELAY = 3
# The download delay setting will honor only one of:
#CONCURRENT_REQUESTS_PER_DOMAIN = 16
#CONCURRENT_REQUESTS_PER_IP = 16

# Disable cookies (enabled by default)
#COOKIES_ENABLED = False

# Disable Telnet Console (enabled by default)
#TELNETCONSOLE_ENABLED = False

# Override the default request headers:
#DEFAULT_REQUEST_HEADERS = {
#   'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
#   'Accept-Language': 'en',
#}

# Enable or disable spider middlewares
# See https://docs.scrapy.org/en/latest/topics/spider-middleware.html
#SPIDER_MIDDLEWARES = {
#    'amazon_scraper.middlewares.AmazonScraperSpiderMiddleware': 543,
#}

# Enable or disable downloader middlewares
# See https://docs.scrapy.org/en/latest/topics/downloader-middleware.html
#DOWNLOADER_MIDDLEWARES = {
#    'amazon_scraper.middlewares.AmazonScraperDownloaderMiddleware': 543,
#}

# Enable or disable extensions
# See https://docs.scrapy.org/en/latest/topics/extensions.html
#EXTENSIONS = {
#    'scrapy.extensions.telnet.TelnetConsole': None,
#}

# Configure item pipelines
# See https://docs.scrapy.org/en/latest/topics/item-pipeline.html
#ITEM_PIPELINES = {
#    'amazon_scraper.pipelines.AmazonScraperPipeline': 300,
#}

# Enable and configure the AutoThrottle extension (disabled by default)
# See https://docs.scrapy.org/en/latest/topics/autothrottle.html
#AUTOTHROTTLE_ENABLED = True
# The initial download delay
#AUTOTHROTTLE_START_DELAY = 5
# The maximum download delay to be set in case of high latencies
#AUTOTHROTTLE_MAX_DELAY = 60
# The average number of requests Scrapy should be sending in parallel to
# each remote server
#AUTOTHROTTLE_TARGET_CONCURRENCY = 1.0
# Enable showing throttling stats for every response received:
#AUTOTHROTTLE_DEBUG = False

# Enable and configure HTTP caching (disabled by default)
# See https://docs.scrapy.org/en/latest/topics/downloader-middleware.html#httpcache-middleware-settings
#HTTPCACHE_ENABLED = True
#HTTPCACHE_EXPIRATION_SECS = 0
#HTTPCACHE_DIR = 'httpcache'
#HTTPCACHE_IGNORE_HTTP_CODES = []
#HTTPCACHE_STORAGE = 'scrapy.extensions.httpcache.FilesystemCacheStorage'

Writing settings.py


In [6]:
%%script bash
mv settings.py scrapers/amazon/amazon_scraper/amazon_scraper/

In [17]:
!pwd

/home/jovyan


In [7]:
%%writefile amazon_spider.py
import scrapy
import numpy as np
import pandas as pd
from scrapy.http import FormRequest

class AmazonSpider(scrapy.Spider):
    name = 'amazon'
    start_urls = ['https://sellercentral.amazon.com']
    data = pd.read_csv('/home/jovyan/store_credentials.csv', header=0, sep=';', index_col=False)
    user = data.Email
    password = data.Password
    
    def get_it(self,response):
        check = response.css('div.active a::attr(href)').get()
        print(check)
        
    def login(self, response):
        token = response.css('form input::attr(value)').get()
        print(token)
        yield FormRequest.from_response(response, formdata = {'appActionToken':token,'email':self.user,'password':self.password},callback=self.get_it)

    def parse(self, response):
        way = response.css('a.link::attr(href)').get()
        yield scrapy.Request(way, self.login)
        #links =\
        #response.css('div.organic__path a::attr(href)').getall()
        #self.url_data = np.append(self.url_data, np.array(links))
        #self.url_counter+=len(links)     
        #next_page = response.css('a.pager__item_kind_next::attr(href)').get()
        #if (self.url_counter<2500) and (next_page is not None):
            #yield scrapy.Request(self.client.scrapyGet(url = self.search_engine+next_page), self.parse)
        
    #def closed(self, reason):
        #print(self.url_counter)
        #raw = pd.DataFrame(data=self.url_data)
        #raw.to_csv(r'/home/jovyan/raw_url.csv', index = False, header=False)

Overwriting amazon_spider.py


In [8]:
%%script bash 
mv amazon_spider.py scrapers/amazon/amazon_scraper/amazon_scraper/spiders

In [9]:
%%script bash
cd scrapers/amazon
. env/bin/activate
cd amazon_scraper/
scrapy crawl amazon

j8QhFewYdNluJr7j2Bb5DttSd5Fggj3D
None


2020-06-05 02:14:03 [scrapy.utils.log] INFO: Scrapy 2.1.0 started (bot: amazon_scraper)
2020-06-05 02:14:03 [scrapy.utils.log] INFO: Versions: lxml 4.5.1.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.7.6 | packaged by conda-forge | (default, Jun  1 2020, 18:57:50) - [GCC 7.5.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 2.9.2, Platform Linux-4.19.104+-x86_64-with-debian-buster-sid
2020-06-05 02:14:03 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2020-06-05 02:14:03 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'amazon_scraper',
 'NEWSPIDER_MODULE': 'amazon_scraper.spiders',
 'SPIDER_MODULES': ['amazon_scraper.spiders'],
 'USER_AGENT': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15) '
               'AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.3 '
               'Safari/605.1.15'}
2020-06-05 02:14:03 [scrapy.extensions.telnet] INFO: Telnet Password: cfd427a642066faa
20